In [35]:
import pandas as pd
import re
import sys
import os

# Añadir la ruta del directorio data_utils al sys.path
# Agregar la carpeta raíz 'data_utils' al sys.path
sys.path.append(os.path.abspath('../'))


In [36]:
#Configuraciones
from data_utils.configuraciones.variables import variables_data

#Manipulacion de datos
from data_utils.manipulacion_de_datos.maniuplacion_de_datos import  procesar_descripcion,   \
                                                                    estandarizado_columnas, \
                                                                    cargar_datos,           \
                                                                    generar_descripcion,    \
                                                                    procesar_descripcion,   \
                                                                    limpiar_datos
from data_utils.manipulacion_de_datos.gestion_de_datos import indexar_productos

#Funciones de Busquedas
from data_utils.funciones_de_busqueda.busqueda_por_filtro import filtrar_por_laboratorio
from data_utils.funciones_de_busqueda.busquedas_por_texto import main_principal

#Aplicaciones de Inteligencia Artificial
from data_utils.utilidades_comunes.aplicaciones_de_inteligencia_artificial.procesamiento_de_lenguaje_natural.index import procesar_comparacion

Instanciamientos de Clases

In [37]:
variables_data = variables_data()


def procesar_descripcion_x(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = texto.strip()

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jeringa prellenada',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prellenada",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda',
            'zyvali x': 'zyvalix',
            'x' : '',
            'lapprell': 'lapicera prellenada',
            'complibprol' : 'comp lib prol ',
            'compcu': 'comp cu',
            'jerprell'  : "jeringa prellenada",
            'prell': 'prellenada'    
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    return data 

Procesamiento df Master Producto

In [38]:
#Carga de Producto
df_Producto = cargar_datos(variables_data.csv_producto)
df_Producto = limpiar_datos(df_Producto)
df_Producto = indexar_productos(df_Producto,variables_data.index_productos)
df_MasterProductos = generar_descripcion(df_Producto,variables_data.name_columna_1_MasterProductos,variables_data.name_columna_2_MasterProductos,variables_data.descripcion_limpia_MasterProductos)
df_MasterProductos = procesar_descripcion_x(df_MasterProductos, variables_data.descripcion_limpia_MasterProductos)
df_MasterProductos['descripcion_limpia_producto']

c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\manipulacion_de_datos\maniuplacion_de_datos.py:15: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(ruta_csv)


0                                                         
1                                                         
2        repetido supportan 500 mililitros caja  15 sac...
3        bolsas papel grado medico 175  65  420 milimet...
4        bolsas papel grado medico 260  60  520 milimet...
                               ...                        
56769              zyvalix 250 miligramos comprimidos  120
56770          zyvalix plus 500 miligramos comprimidos  60
56771    zyvox iv 600 miligramos bolsas  10  300 milili...
56772                zyvox oral 600 miligramos tableta  10
56773                               zz top comprimidos  30
Name: descripcion_limpia_producto, Length: 56770, dtype: object

DataFrame Master Producto Filtrado por Laboratorio

In [39]:
# El usuario ingresa el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ").strip()
# Filtrar productos por laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)

Se encontraron 38 productos coincidentes del laboratorio 'ABBVIE'.


DataFrame Proveedor

In [40]:
#Carga Proveedor
cvs_proveedor = 'ABBVIE.csv'
df_Proveedor = cargar_datos(cvs_proveedor)
# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = limpiar_datos(df_Proveedor)
#Indexacion 
df_Proveedor = indexar_productos(df_Proveedor,variables_data.index_proveedor)
#Normalizacion de Columnas
df_Proveedor = estandarizado_columnas(df_Proveedor,variables_data.name_columna_1_Proveedor,variables_data.name_columna_2_Proveedor,variables_data.descripcion_limpia_Proveedor)
#Normalizacion de Columna Descripcion
df_Proveedor = procesar_descripcion_x(df_Proveedor, variables_data.descripcion_limpia_Proveedor)
df_Proveedor.columns

#resultado_df = pd.DataFrame(df_Proveedor)
#resultado_df.to_csv("df_Proveedor.csv", index=False)

Index(['condición de precios', 'cod. producto', 'descripcion', 'presentacion',
       'unidades ', 'precio s/iva', 'descuento', 'total', 'index_proveedor',
       'descripcion_limpia_proveedor'],
      dtype='object')

Procesamiento CODIGO DE PRODUCTO

In [41]:
df_MasterProductos_Filtro.columns
# Guardar resultados en CSV
#resultado_df = pd.DataFrame(df_MasterProductos_Filtro)
#resultado_df.to_csv("df_MasterProductos_Filtro.csv", index=False)

Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto'],
      dtype='object')

In [42]:
df_Proveedor.head(10)

,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor
0,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellena...
1,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellen...
2,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellena...
3,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",3,rinvoq 15 miligramos 15 miligramos 30 comprim...
4,NaN,20069005,RINVOQ 30mg,30 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,4,rinvoq 30 miligramos 30 miligramos 30 comprim...
5,NaN,20078725,RINVOQ 45mg,45 MG x 30 COMP REC DE LIB. PROLONGADA,NaN,NaN,NaN,-,5,rinvoq 45 miligramos 45 miligramos 30 comprim...
6,NaN,20069017,SKYRIZI 150mg,1 LAPICERA PRELLENADA por 1 ML (150 mg),NaN,NaN,NaN,-,6,skyrizi 150 miligramos 1 lapicera prellenada p...
7,NaN,20079636,SKYRIZI 360mg,2.4ML OBI AR FP,NaN,NaN,NaN,-,7,skyrizi 360 miligramos 24 mililitros obi ar fp
8,NaN,20079635,SKYRIZI 600 mg,10.0 mL VIAL AR FP,NaN,NaN,NaN,-,8,skyrizi 600 miligramos 100 mililitros vial ar fp
9,NaN,20011756,MAVIRET,100 MG/40 MG x 84 COMP REC,NaN,NaN,NaN,-,9,maviret 100 miligramos 40 miligramos 84 compr...


Procesamiento NLP DESCRIPCION

In [43]:
df_resultado_similiridad = main_principal(df_MasterProductos_Filtro, df_Proveedor)
#df_resultado_similiridad

Procesamiento para comparacion de  Descripciones

In [44]:
df_resultado_similiridad[df_resultado_similiridad['similaridad'] > 0.8][['similaridad', 'niprod', 'cod_prod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'index_proveedor_y','codbarra']]

,similaridad,niprod,cod_prod,descripcion_limpia_proveedor,descripcion_limpia_producto,index_proveedor_y,codbarra
2,0.944039,119550.0,55604,humira ac 1 80 miligramos 1 lapicera prellena...,humira ac 1 80 miligramos 80 miligramos 08 mi...,2,8054083017648
6,0.876686,124898.0,59720,skyrizi 150 miligramos 1 lapicera prellenada p...,skyrizi 150 miligramos lapicera prellenada 1,6,8054083021935.0
10,0.852467,119575.0,53119,venclexta kit inicio kit de inicio 1 pack 4 ...,venclexta kit de inicio 1,10,8054083013343.0
11,0.943522,119573.0,53120,venclexta 100 miligramos 100 miligramos 120 c...,venclexta 100 miligramos comprimidos 120,11,8054083013336.0
14,0.812576,119561.0,14916,sevorane 250 mililitros,sevorane envase 250 mililitros,14,8054083015156.0
17,1.000000,119577.0,44168,zemplar 2 microgramo 30 capsulas,zemplar 2 microgramo capsulas 30,17,8054083012032.0


INDEX DE SUSTRACCION PARA EL DF PROVEEDOR

In [45]:
#Sutraccion de productos sobre df ORIGINAL DE PROVEEDORES.
similiridad = 'similaridad'

umbral_similaridad = 0.9  # Ajusta este valor según tu caso
df_filtrado_similaridad = df_resultado_similiridad[df_resultado_similiridad[similiridad] > umbral_similaridad]

# Ahora, con los índices originales, extraemos los productos del df_Proveedor
# Filtramos por 'indice_original_proveedor' para obtener solo las filas que superaron el umbral
df_Proveedor_Similiridad_filtrado = df_Proveedor[df_Proveedor[variables_data.index_proveedor].isin(df_filtrado_similaridad['index_proveedor_x'])]

# Mostrar los productos filtrados del proveedor
df_Proveedor_Similiridad_filtrado


,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor
2,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellena...
11,NaN,20019067,VENCLEXTA 100mg,100 MG X 120 COMP.,NaN,NaN,NaN,NaN,11,venclexta 100 miligramos 100 miligramos 120 c...
17,NaN,20059449,ZEMPLAR 2 MCG,30 CAPSULAS,NaN,NaN,NaN,-,17,zemplar 2 microgramo 30 capsulas


In [46]:
df_MasterProductos.columns

#Arquitectura df_Productos
 
# Niprod = Sistema Carena
# ean = ????
# descrip1 = Descripcion Provedor
# descrip2 = Presentacion Proveedor
# atrib0   = laboratorio
# codbarra = codbarra
#cod_prod  = codigo_producto_laboratorio

Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto'],
      dtype='object')

MODELO DE PROCESAMIENTOS DE LENGUAJE NATURAL

Bert

In [47]:

modelo_nombre_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

resultados_comparaciones = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BERT,                # modelo_nombre
    "BERT",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\utilidades_comunes\aplicaciones_de_inteligencia_artificial\procesamiento_de_lenguaje_natural\index.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_master['numeros'] = df_master[columna_master].apply(extraer_numeros)  # Extrae números del master
c:\Users\Windows\Desktop\digitalizacion_distriapp\backend\src\data_utils\utilidades_comunes\aplicaciones_de_inteligencia_a

In [48]:
resultados_comparaciones.head(10)


,descripcion_proveedor,descripcion_match,similitud,index_proveedor,index_producto,niprod
0,humira ac 2 20 miligramos 2 jeringas prellena...,humira ac 2 20 miligramos 20 miligramos 02 mi...,0.901277,0,27614,119551.0
1,humira ac 2 20 miligramos 2 jeringas prellena...,humira jeringa prellenada 2 08 mililitros,0.821588,0,27616,119549.0
2,humira ac 2 20 miligramos 2 jeringas prellena...,skyrizi 75 miligramos jeringa prellenada 2,0.801296,0,47492,119565.0
3,humira ac 2 40 miligramos 2 lapiceras prellen...,humira ac 2 40 miligramos 40 miligramos 04 mi...,0.862091,1,27615,119552.0
4,humira ac 2 40 miligramos 2 lapiceras prellen...,humira pen lapicera 2 08 mililitros,0.811453,1,27618,119554.0
5,humira ac 2 40 miligramos 2 lapiceras prellen...,humira jeringa prellenada 2 08 mililitros,0.808174,1,27616,119549.0
6,humira ac 1 80 miligramos 1 lapicera prellena...,humira ac 1 80 miligramos 80 miligramos 08 mi...,0.907117,2,27613,119550.0
7,humira ac 1 80 miligramos 1 lapicera prellena...,humira pen lapicera 2 08 mililitros,0.812178,2,27618,119554.0
8,humira ac 1 80 miligramos 1 lapicera prellena...,humira jeringa prellenada 2 08 mililitros,0.810118,2,27616,119549.0
9,rinvoq 15 miligramos 15 miligramos 30 comprim...,rinvoq 15 miligramos comp lib prol 30,0.862737,3,44810,119559.0


In [49]:
# Unir df_Proveedor con resultados_comparaciones en base a index_proveedor
df_resultado = df_Proveedor.merge(resultados_comparaciones, 
                                  left_index=True,  # Índice de df_Proveedor (index_proveedor debe coincidir con el índice)
                                  right_on='index_proveedor',  # Columna en resultados_comparaciones
                                  how='right')  # Solo los que están en resultados_comparaciones

# Reorganizar las columnas si es necesario
df_resultado = df_resultado[df_Proveedor.columns.tolist() + ['index_producto', 'niprod', 'similitud']]

# Mostrar el resultado
df_resultado.head(15)


,condición de precios,cod. producto,descripcion,presentacion,unidades,precio s/iva,descuento,total,index_proveedor,descripcion_limpia_proveedor,numeros,index_producto,niprod,similitud
0,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellena...,"[2, 20, 2, 02]",27614,119551.0,0.901277
1,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellena...,"[2, 20, 2, 02]",27616,119549.0,0.821588
2,NaN,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,-,0,humira ac 2 20 miligramos 2 jeringas prellena...,"[2, 20, 2, 02]",47492,119565.0,0.801296
3,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellen...,"[2, 40, 2, 04]",27615,119552.0,0.862091
4,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellen...,"[2, 40, 2, 04]",27618,119554.0,0.811453
5,NaN,20005289,HUMIRA AC x 2 (40mg),"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,-,1,humira ac 2 40 miligramos 2 lapiceras prellen...,"[2, 40, 2, 04]",27616,119549.0,0.808174
6,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellena...,"[1, 80, 1, 08]",27613,119550.0,0.907117
7,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellena...,"[1, 80, 1, 08]",27618,119554.0,0.812178
8,NaN,20021980,HUMIRA AC x 1 (80 mg),"1 LAPICERA PRELL. X 0,8 ml",NaN,NaN,NaN,-,2,humira ac 1 80 miligramos 1 lapicera prellena...,"[1, 80, 1, 08]",27616,119549.0,0.810118
9,18% DESCUENTO -ISJ,20062091,RINVOQ 15mg,15 MG x 30 COMP REC DE LIB. PROLONGADA,1.0,"2.345.085,38",18%,"1.987.360,49",3,rinvoq 15 miligramos 15 miligramos 30 comprim...,"[15, 15, 30]",44810,119559.0,0.862737


In [58]:
# Unir df_resultado con masterProductos en base a la columna niprod
df_resultado = df_resultado.merge(df_MasterProductos_Filtro[['niprod', variables_data.descripcion_limpia_MasterProductos]], 
                                  on='niprod',  # Clave de unión
                                  how='left')  # 'left' para no perder filas en df_resultado

# Mostrar el resultado con la nueva columna agregada
df_resultado[['descripcion_limpia_proveedor','niprod','index_producto','similitud','index_proveedor']]

niprod
#Arquitectura df_Productos
 
# Niprod = Sistema Carena
# ean = ????
# descrip1 = Descripcion Provedor
# descrip2 = Presentacion Proveedor
# atrib0   = laboratorio
# codbarra = codbarra
#cod_prod  = codigo_producto_laboratorio

,descripcion_limpia_proveedor,niprod,index_producto,similitud,index_proveedor
0,humira ac 2 20 miligramos 2 jeringas prellena...,119551.0,27614,0.901277,0
1,humira ac 2 20 miligramos 2 jeringas prellena...,119549.0,27616,0.821588,0
2,humira ac 2 20 miligramos 2 jeringas prellena...,119565.0,47492,0.801296,0
3,humira ac 2 40 miligramos 2 lapiceras prellen...,119552.0,27615,0.862091,1
4,humira ac 2 40 miligramos 2 lapiceras prellen...,119554.0,27618,0.811453,1
...,...,...,...,...,...
58,boto vial 200 unidad,119577.0,56457,0.730224,19
59,boto vial 200 unidad,119567.0,49698,0.721654,19
60,ozurde 07 miligramos c 1,130998.0,47491,0.851499,20
61,ozurde 07 miligramos c 1,119568.0,49819,0.838913,20


PubMedBERT

In [51]:
"""
modelo_nombre_PubMedBert = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

resultados_PubMedBERT = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_PubMedBert,                # modelo_nombre
    "PubMedBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
"""

'\nmodelo_nombre_PubMedBert = \'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract\'\n\nresultados_PubMedBERT = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_PubMedBert,                # modelo_nombre\n    "PubMedBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n'

BioBert

In [52]:
"""
modelo_nombre_BioBert = 'dmis-lab/biobert-base-cased-v1.1'

resultados_BioBert = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BioBert,                # modelo_nombre
    "BioBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
 """

'\nmodelo_nombre_BioBert = \'dmis-lab/biobert-base-cased-v1.1\'\n\nresultados_BioBert = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_BioBert,                # modelo_nombre\n    "BioBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n '

FINE TUNING